# Base Variables

In [1]:
# average / weighted_average / bitwise_or
ENSEMBLE_METHOD = "average"
VIABLE_COUNTABILITY = 0
AVERAGE_ACCEPTABILITY = 0
# Minimum value of credibility per mask
CREDIBILITY_THRESHOLD = 0.6
# Minimum IoU in order to group instances together
DEVIATION_THRESHOLD = 0.5
# Minimum IoU in order to compare instances while evaluating 
DEVIATION_THRESHOLD_EVAL = 0.5
#For Weighted Average
AVERAGE_ACCEPTABILITY_2 = 0.5

# Segmentation Model Dictionary

In [2]:
import os

model_dict = []
model_dict.append(('hybrid_task_cascade_mask_rcnn_X-101-64x4d-FPN',('configs/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco.py',
                                                                'checkpoints/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_20200312-946fd751.pth',
                                                                'https://download.openmmlab.com/mmdetection/v2.0/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_20200312-946fd751.pth')))
model_dict.append(('detectors_htc_r101_20e_coco',('configs/detectors/detectors_htc_r101_20e_coco.py',
                                                  'checkpoints/detectors_htc_r101_20e_coco_20210419_203638-348d533b.pth',
                                                  'https://download.openmmlab.com/mmdetection/v2.0/detectors/detectors_htc_r101_20e_coco/detectors_htc_r101_20e_coco_20210419_203638-348d533b.pth')))
model_dict.append(('cascade_mask_rcnn_X-101-64x4d-FPN',('configs/cascade_rcnn/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco.py',
                                                        'checkpoints/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco_20210719_210311-d3e64ba0.pth',
                                                        'https://download.openmmlab.com/mmdetection/v2.0/cascade_rcnn/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco/cascade_mask_rcnn_x101_64x4d_fpn_mstrain_3x_coco_20210719_210311-d3e64ba0.pth')))
model_dict.append(('cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco',('configs/dcn/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py',
                                                                           'checkpoints/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco-e75f90c8.pth',
                                                                           'https://download.openmmlab.com/mmdetection/v2.0/dcn/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco/cascade_mask_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco-e75f90c8.pth')))
model_dict.append(('gcnet_X-101-FPN_DCN_Cascade_Mask_GC(c3-c5,r4)',('configs/gcnet/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco.py',
                                                                    'checkpoints/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco_20210615_161851-720338ec.pth',
                                                                    'https://download.openmmlab.com/mmdetection/v2.0/gcnet/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco/cascade_mask_rcnn_x101_32x4d_fpn_syncbn-backbone_dconv_c3-c5_r4_gcb_c3-c5_1x_coco_20210615_161851-720338ec.pth')))

key_list = [model[0] for model in model_dict]

#test_config = 'configs/common/mstrain-poly_3x_coco_instance.py'
test_config = 'configs/_base_/datasets/cityscapes_instance.py'
dataset_name = os.path.splitext(test_config)[0].split('/')[-1]

# Auxiliary Functions

In [3]:
import numpy as np
import torch

def IoU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def IoU_Mask(maskA, maskB):
    intersection = np.logical_and(maskA, maskB).astype(np.uint8)
    union = np.logical_or(maskA, maskB).astype(np.uint8)
    iou = np.sum(intersection)/np.sum(union)
    return iou

# Ensemble Network

In [4]:
BATCH_SIZE = 1
N_CHANNELS = 5

In [5]:
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import torch

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.double_conv(x)
    
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)
    
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [7]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = UNet(N_CHANNELS,1).float().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load("work_dirs/u_net_ensemble_network_no_img_0.001_lr/epoch_50.pt")
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
net.eval()

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(5, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [8]:
from torchinfo import summary

summary(net, (4,5,572,572))

/home/yusoi/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                        Output Shape              Param #
UNet                                          --                        --
├─DoubleConv: 1-1                             [4, 64, 572, 572]         --
│    └─Sequential: 2-1                        [4, 64, 572, 572]         --
│    │    └─Conv2d: 3-1                       [4, 64, 572, 572]         2,944
│    │    └─BatchNorm2d: 3-2                  [4, 64, 572, 572]         128
│    │    └─ReLU: 3-3                         [4, 64, 572, 572]         --
│    │    └─Conv2d: 3-4                       [4, 64, 572, 572]         36,928
│    │    └─BatchNorm2d: 3-5                  [4, 64, 572, 572]         128
│    │    └─ReLU: 3-6                         [4, 64, 572, 572]         --
├─Down: 1-2                                   [4, 128, 286, 286]        --
│    └─Sequential: 2-2                        [4, 128, 286, 286]        --
│    │    └─MaxPool2d: 3-7                    [4, 64, 286, 286]         --
│    │    └

In [19]:
from skimage.transform import resize
from torchvision import transforms, utils

class Resize(object):
    def __init__(self, size):
        self.size = size

    def __call__(self,sample):
        
        return resize(sample,(*self.size,N_CHANNELS))

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):

        # torch image: C x H x W
        sample = sample.transpose((2, 0, 1))
        return torch.from_numpy(sample)


# Ensemble Functions

In [14]:
from mmcv import Config
from mmdet.datasets import build_dataset, build_dataloader
from mmcv.parallel import MMDataParallel
from mmdet.apis import single_gpu_test
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from typing import List, Tuple, Union, Dict
from torch import nn
from os import path
from urllib import request
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from skimage.transform import resize
from torchvision import transforms, utils

import mmcv
import os.path as osp
import pycocotools.mask as mask_util

COCO_CLASSES = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
                'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
                'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
                'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
                'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
                'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
                'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')

CITYSCAPES_CLASSES = ('person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle',
                      'bicycle')

WORK_DIR = "work_dirs/ensemble_results/"

def get_dataset():
    test_cfg = Config.fromfile(test_config)
    # in case the test dataset is concatenated
    samples_per_gpu = 1
    if isinstance(test_cfg.data.test, dict):
        test_cfg.data.test.test_mode = True
        samples_per_gpu = test_cfg.data.test.pop('samples_per_gpu', 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            test_cfg.data.test.pipeline = replace_ImageToTensor(
                test_cfg.data.test.pipeline)
    elif isinstance(test_cfg.data.test, list):
        for ds_cfg in test_cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in test_cfg.data.test])
        if samples_per_gpu > 1:
            for ds_cfg in test_cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    # init distributed env first, since logger depends on the dist info.
    distributed = False

    #rank, _ = get_dist_info()
    # allows not to create
    #mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
    #timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    #json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(test_cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=test_cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)
    
    return dataset, data_loader

def inference_on_dataset(model_info):
    config, checkpoint = model_info
    
    cfg = Config.fromfile(config)

    if cfg.get('custom_imports', None):
        from mmcv.utils import import_modules_from_strings
        import_modules_from_strings(**cfg['custom_imports'])
    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True

    cfg.model.pretrained = None
    if cfg.model.get('neck'):
        if isinstance(cfg.model.neck, list):
            for neck_cfg in cfg.model.neck:
                if neck_cfg.get('rfp_backbone'):
                    if neck_cfg.rfp_backbone.get('pretrained'):
                        neck_cfg.rfp_backbone.pretrained = None
        elif cfg.model.neck.get('rfp_backbone'):
            if cfg.model.neck.rfp_backbone.get('pretrained'):
                cfg.model.neck.rfp_backbone.pretrained = None
                
    test_cfg = Config.fromfile(test_config)
    # in case the test dataset is concatenated
    samples_per_gpu = 1
    if isinstance(cfg.data.test, dict):
        test_cfg.data.test.test_mode = True
        samples_per_gpu = test_cfg.data.test.pop('samples_per_gpu', 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            test_cfg.data.test.pipeline = replace_ImageToTensor(
                test_cfg.data.test.pipeline)
    elif isinstance(test_cfg.data.test, list):
        for ds_cfg in test_cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in test_cfg.data.test])
        if samples_per_gpu > 1:
            for ds_cfg in test_cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    # init distributed env first, since logger depends on the dist info.
    distributed = False

    #rank, _ = get_dist_info()
    # allows not to create
    #mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
    #timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    #json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(test_cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=test_cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)

    # build the model and load checkpoint
    cfg.model.train_cfg = None
    model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        wrap_fp16_model(model)
    checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
    #if args.fuse_conv_bn:
        #model = fuse_conv_bn(model)
    # old versions did not save class info in checkpoints, this walkaround is
    # for backward compatibility
    if 'CLASSES' in checkpoint.get('meta', {}):
        model.CLASSES = checkpoint['meta']['CLASSES']
    else:
        model.CLASSES = dataset.CLASSES

    #classes = model.CLASSES.copy()
    classes = list(enumerate(model.CLASSES)).copy()
    model = MMDataParallel(model, device_ids=[0])
    outputs = single_gpu_test(model, data_loader, None, None, None)
    
    print(len(dataset))
    
    return outputs, classes, len(dataset)


def gather_results_from_model(model_name: str, model_info: Tuple[str,str], score_thr:float, person_only:bool , result_type = 'bbox'):
    if not osp.exists(WORK_DIR+model_name+"_"+dataset_name+".pkl"):
        mmcv.mkdir_or_exist(osp.abspath(WORK_DIR))
        results, original_classes, dataset_size = inference_on_dataset(model_info)
        classes = original_classes
        mmcv.dump(results, WORK_DIR+model_name+"_"+dataset_name+".pkl")
    else:
        config,checkpoint = model_info
        cfg = Config.fromfile(config)
        model = build_detector(cfg.model)
        checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
        if 'CLASSES' in checkpoint.get('meta', {}):
            model.CLASSES = checkpoint['meta']['CLASSES']
        else:
            model.CLASSES = dataset.CLASSES
        original_classes = list(enumerate(model.CLASSES)).copy()
        classes = original_classes
        results = mmcv.load(WORK_DIR+model_name+"_"+dataset_name+".pkl")
        dataset_size = len(results)
    if person_only:
        if len(classes) == len(COCO_CLASSES):
            classes = [(0,'person')]
        elif len(classes) == len(CITYSCAPES_CLASSES):
            classes = [(0,'person'),(1,'rider')]
    
        
    return results,classes,dataset_size

def gather_results(model_dict: Dict[str,Tuple[str,str,str]], score_thr: float, person_only: bool, result_type='bbox'):
    #model_dict = model_dict.items()
    ensemble_results = {}
    dataset_compatible = -1
    label_type = []
    for i, (name, (config,checkpoint,download_link)) in enumerate(model_dict):
        if not path.exists(checkpoint):
            print("Downloading",name)
            request.urlretrieve(download_link,checkpoint)
            print("Finished downloading",name)
        print("Loading inference results from model:",name)
        ensemble_results[i],classes,dataset_size = gather_results_from_model(name, (config,checkpoint), score_thr, person_only, result_type)
        label_type.append(len(classes))
        if dataset_compatible < 0 or dataset_compatible == dataset_size:
            dataset_compatible = dataset_size
        else:
            raise(Exception("Dataset sizes are not compatible"))
    return ensemble_results,classes,dataset_compatible

def group_instances(dataset,model_dict,ensemble_results, labels: List[str], dataset_size, score_thr, threshold, ensemble_method):
    #ensemble_results[model][image][bbox or segm][label][instance]
    final_results = []
    n_models = len(ensemble_results)
    #Iterate over all the images
    for img in tqdm(range(0,len(dataset))):
        bbox_group = []
        segm_group = []
        
        if ensemble_method == "network":
            filename = dataset[img]['img_metas'][0].data['filename']
            ori_img_size = dataset[img]['img_metas'][0].data['ori_shape']
            transform = transforms.Compose([Resize((572,572)),ToTensor()])
            image = Image.open(filename)
            img_array = np.asarray(image)

        #Iterate over all the labels
        for (label_nr,label) in labels:
            bbox_results = []
            segm_results = []
            #Create a matrix of already used instances
            used_instances = []
            for cur_model in range(0,len(ensemble_results)):
                used_instances.insert(cur_model,[False]*len(ensemble_results[cur_model][img][0][label_nr]))
                
            #Iterate over all the models for a certain label and a certain image
            for cur_model in range(0,len(ensemble_results)):
                #Iterate over the current model's results on a certain label on a certain image
                for cur_instance in range(0,len(ensemble_results[cur_model][img][0][label_nr])):
                    if not used_instances[cur_model][cur_instance] and ensemble_results[cur_model][img][0][label_nr][cur_instance][4] >= CREDIBILITY_THRESHOLD:
                    #if not used_instances[cur_model][cur_instance] and ensemble_results[cur_model][img][0][label_nr][cur_instance][4] >= model_dict[cur_model][1][3]:
                        used_instances[cur_model][cur_instance] = True
                        cur_instance_group = [None for w in range(0,len(ensemble_results))]
                        cur_instance_group[cur_model] = (ensemble_results[cur_model][img][0][label_nr][cur_instance],
                                                         ensemble_results[cur_model][img][1][label_nr][cur_instance])
                        #Iterate over all the other models
                        for comp_model in range(cur_model+1,len(ensemble_results)):
                            deviations = []
                            #Iterate over each of the other model's results
                            for comp_instance in range(0,len(ensemble_results[comp_model][img][0][label_nr])):
                                if ensemble_results[comp_model][img][0][label_nr][comp_instance][4] >= CREDIBILITY_THRESHOLD:
                                    if not used_instances[comp_model][comp_instance]:
                                        #cur_iou = IoU(ensemble_results[cur_model][img][0][label_nr][cur_instance],ensemble_results[comp_model][img][0][label_nr][comp_instance])
                                        boxA = ensemble_results[cur_model][img][0][label_nr][cur_instance]
                                        boxB = ensemble_results[comp_model][img][0][label_nr][comp_instance]
                                        xA = int(round(min(boxA[0], boxB[0])))
                                        yA = int(round(min(boxA[1], boxB[1])))
                                        xB = int(round(max(boxA[2], boxB[2])))
                                        yB = int(round(max(boxA[3], boxB[3])))
                                        cur_iou = IoU_Mask(mask_util.decode(ensemble_results[cur_model][img][1][label_nr][cur_instance])[yA:yB,xA:xB],
                                                           mask_util.decode(ensemble_results[comp_model][img][1][label_nr][comp_instance])[yA:yB,xA:xB])
                                        
                                    else:
                                        cur_iou = 0.0
                                    deviations.append(cur_iou)
                            #Check if the max iou is within the threshold and add the new instance to the group
                            if len(deviations) > 0:
                                pos = max(range(len(deviations)), key=deviations.__getitem__)
                                if deviations[pos] >= threshold:
                                    #Guarantee this instance isn't used again
                                    used_instances[comp_model][pos] = True
                                    cur_instance_group[comp_model] = (ensemble_results[comp_model][img][0][label_nr][pos],
                                                                      ensemble_results[comp_model][img][1][label_nr][pos])
                        
                        count = 0
                        for instance_i in cur_instance_group:
                            if instance_i:
                                count += 1
                                
                        # Assuming an instance group is viable if most of the networks identified it
                        if (count >= (n_models/2) + VIABLE_COUNTABILITY and (not (ensemble_method == "bitwise_and")) and (not (ensemble_method == "bitwise_or"))) or \
                           (count == n_models and ensemble_method == "bitwise_and") or \
                           (ensemble_method == "bitwise_or"):
                            bbox = np.array([0.0]*5)
                            for model_result in range(0,len(cur_instance_group)):
                                if not cur_instance_group[model_result] is None:
                                    bbox = np.add(bbox,cur_instance_group[model_result][0])
                            bbox = (bbox/count)
                            confidence = bbox[4]
                            bbox = bbox.astype(int)
                            bbox[0:3] = np.around(bbox[0:3])
                            bbox_y = (bbox[3]-bbox[1]).astype(int)
                            bbox_x = (bbox[2]-bbox[0]).astype(int)
                            if ensemble_method == "network":
                                return_group = []
                                for x in range(len(cur_instance_group)):
                                    if cur_instance_group[x] is None:
                                        return_group.append(np.zeros(ori_img_size,dtype=np.uint8))
                                    else:
                                        return_group.append(mask_util.decode(cur_instance_group[x][1]))
                                pred_stack = np.dstack(return_group)
                                #network_input = transform(np.dstack((img_array,pred_stack)))[None,:].float().to(device)
                                network_input = transform(pred_stack)[None,:].float().to(device)
                                mask = net(network_input)

                                mask = mask.cpu().detach()
                                mask = mask.numpy().squeeze(axis=0).transpose((1,2,0)).squeeze(axis=2)
                                mask = mask > -7

                                fig = plt.figure()
                                plt.imshow(mask,cmap='gray')
                                plt.show()
                                img_size = (ori_img_size[0],ori_img_size[1])
                                segmentation = mask.astype("uint8")
                                
                            else:
                                mask = np.zeros((bbox_y,bbox_x),dtype=int)
                                img_size = (0,0)
                                if ensemble_method == "average":
                                    for model_result in range(0,len(cur_instance_group)):
                                        if not cur_instance_group[model_result] is None:
                                            decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                            mask = mask+decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int)
                                            img_size = decoded_mask.shape
                                    acceptability = max(1,count/2 + AVERAGE_ACCEPTABILITY)
                                    mask = mask >= acceptability
                                elif ensemble_method == "weighted_average":
                                    total_confidence = 0.0
                                    for model_result in range(0,len(cur_instance_group)):
                                        if not cur_instance_group[model_result] is None:
                                            decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                            mask = mask+(decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int) * confidence)
                                            total_confidence += confidence
                                            img_size = decoded_mask.shape
                                    mask = mask >= AVERAGE_ACCEPTABILITY_2 * total_confidence
                                elif ensemble_method == "bitwise_or":
                                    for model_result in range(0,len(cur_instance_group)):
                                        if not cur_instance_group[model_result] is None:
                                            decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                            mask = mask+decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int)
                                            img_size = decoded_mask.shape
                                    mask = mask > 0.0
                                elif ensemble_method == "bitwise_and":
                                    for model_result in range(0,len(cur_instance_group)):
                                        decoded_mask = mask_util.decode(cur_instance_group[model_result][1])
                                        mask = mask+decoded_mask[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x].astype(int)
                                        img_size = decoded_mask.shape
                                    mask = mask == float(n_models)
                                    
                                segmentation = np.zeros(img_size).astype(bool)
                                segmentation[bbox[1]:bbox[1]+bbox_y,bbox[0]:bbox[0]+bbox_x] = mask
                            
                            
                            bbox = bbox.astype(float)
                            bbox[4] = confidence
                            bbox_results.append(np.array(bbox))
                            segm_results.append(mask_util.encode(np.asfortranarray(segmentation)))
                            #segm_results.append(np.array(segmentation))
            if not bbox_results is None:
                np.append(bbox_results,np.array([]))  
            bbox_group.append(np.array(bbox_results).reshape(-1,5)) 
            segm_group.append(segm_results)
        final_results.append((bbox_group,segm_group))            
                            
    return final_results



def run_ensemble(dataset, model_dict: Dict[str,Tuple[str,str,str]], score_thr: float, person_only: bool, ensemble_method: str, result_type='segm'):
    ensemble_results,classes,dataset_size = gather_results(model_dict,score_thr,person_only,result_type)
    results = group_instances(dataset,model_dict,ensemble_results,classes,dataset_size,score_thr,DEVIATION_THRESHOLD,ensemble_method)
    #Force garbage collection in order to release memory
    return results

# Run Ensemble

In [15]:
import glob
import warnings
import gc
import os
import pickle

warnings.filterwarnings('ignore')
dataset,dataloader = get_dataset()
results = run_ensemble(dataset,model_dict, CREDIBILITY_THRESHOLD,True,ENSEMBLE_METHOD,result_type='segm')
#results = get_dataset(model_dict, CREDIBILITY_THRESHOLD,True,ENSEMBLE_METHOD,result_type='segm')
warnings.filterwarnings('default')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading inference results from model: hybrid_task_cascade_mask_rcnn_X-101-64x4d-FPN
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: checkpoints/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_20200312-946fd751.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1525/1525, 1.2 task/s, elapsed: 1253s, ETA:     0s1525
Loading inference results from model: detectors_htc_r101_20e_coco
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: checkpoints/detectors_htc_r101_20e_coco_20210419_203638-348d533b.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1525/1525, 1.2 task/s, elapsed: 1315s, ETA:     0s1525
Loading inference results from model: cascade_mask_rcnn_X-101-64x4d-FPN
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: checkpoints/ca

  8%|███                                     | 119/1525 [00:34<06:45,  3.46it/s]


KeyboardInterrupt: 

In [9]:
checkpoint['loss']

NameError: name 'checkpoint' is not defined

In [ ]:
dataset,dataloader = get_dataset()
print(dataset[0])

# Evaluation

In [10]:
test_cfg = Config.fromfile(test_config)
# in case the test dataset is concatenated
samples_per_gpu = 1
if isinstance(test_cfg.data.test, dict):
    test_cfg.data.test.test_mode = True
    samples_per_gpu = test_cfg.data.test.pop('samples_per_gpu', 1)
    if samples_per_gpu > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        test_cfg.data.test.pipeline = replace_ImageToTensor(
            test_cfg.data.test.pipeline)
elif isinstance(test_cfg.data.test, list):
    for ds_cfg in test_cfg.data.test:
        ds_cfg.test_mode = True
    samples_per_gpu = max(
        [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in test_cfg.data.test])
    if samples_per_gpu > 1:
        for ds_cfg in test_cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

# init distributed env first, since logger depends on the dist info.
distributed = False

#rank, _ = get_dist_info()
# allows not to create
#mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
#timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
#json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

# build the dataloader
dataset = build_dataset(test_cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=samples_per_gpu,
    workers_per_gpu=test_cfg.data.workers_per_gpu,
    dist=distributed,
    shuffle=False)

#print(metric)
#metric_dict = dict(config=args.config, metric=metric)
#if args.work_dir is not None and rank == 0:
    #mmcv.dump(metric_dict, json_file)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [ ]:
print(dataset)

In [ ]:
eval_kwargs = dict(metric=["bbox","segm"],classwise=True)
metric = dataset.evaluate(results, **eval_kwargs)

In [11]:
bc_info = dataset.custom_evaluate(results,DEVIATION_THRESHOLD_EVAL)

print("TP","{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[0]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[1]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[2]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[3]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[4]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[5]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[6]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tp'].values()) for j in sub[7]])))
print("FP","{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[0]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[1]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[2]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[3]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[4]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[5]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[6]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fp'].values()) for j in sub[7]])))
print("TN","{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[0]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[1]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[2]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[3]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[4]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[5]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[6]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['tn'].values()) for j in sub[7]])))
print("FN","{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[0]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[1]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[2]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[3]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[4]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[5]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[6]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['fn'].values()) for j in sub[7]])))

print("P","{:.5f}".format(np.average([j for sub in list(bc_info['p'].values()) for j in sub[0]])),
          "{:.5f}".format(np.average([j for sub in list(bc_info['p'].values()) for j in sub[1]])),
          "{:.5f}".format(np.average([j for sub in list(bc_info['p'].values()) for j in sub[2]])),
          "{:.5f}".format(np.average([j for sub in list(bc_info['p'].values()) for j in sub[3]])))
print("R","{:.5f}".format(np.average([j for sub in list(bc_info['r'].values()) for j in sub[0]])),
          "{:.5f}".format(np.average([j for sub in list(bc_info['r'].values()) for j in sub[1]])),
          "{:.5f}".format(np.average([j for sub in list(bc_info['r'].values()) for j in sub[2]])),
          "{:.5f}".format(np.average([j for sub in list(bc_info['r'].values()) for j in sub[3]])))
print("F1","{:.5f}".format(np.average([j for sub in list(bc_info['f1'].values()) for j in sub[0]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['f1'].values()) for j in sub[1]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['f1'].values()) for j in sub[2]])),
           "{:.5f}".format(np.average([j for sub in list(bc_info['f1'].values()) for j in sub[3]])))

cg_list = list(bc_info['cg'].values())
ig_list = list(bc_info['ig'].values())
ng_list = list(bc_info['ng'].values())
cg = np.sum(cg_list,axis=0)
ig = np.sum(ig_list,axis=0)
ng = np.sum(ng_list,axis=0)

total_guesses = sum([cg[0],ig[0],ng[0]])
total_guesses_small = sum([cg[1],ig[1],ng[1]])
total_guesses_medium = sum([cg[2],ig[2],ng[2]])
total_guesses_large = sum([cg[3],ig[3],ng[3]])

print("CG",cg[0],"{:.5f}".format(cg[0]/total_guesses),
           cg[1],"{:.5f}".format(cg[1]/total_guesses_small),
           cg[2],"{:.5f}".format(cg[2]/total_guesses_medium),
           cg[3],"{:.5f}".format(cg[3]/total_guesses_large))
print("IG",ig[0],"{:.5f}".format(ig[0]/total_guesses),
           ig[1],"{:.5f}".format(ig[1]/total_guesses_small),
           ig[2],"{:.5f}".format(ig[2]/total_guesses_medium),
           ig[3],"{:.5f}".format(ig[3]/total_guesses_large))
print("NG",ng[0],"{:.5f}".format(ng[0]/total_guesses),
           ng[1],"{:.5f}".format(ng[1]/total_guesses_small),
           ng[2],"{:.5f}".format(ng[2]/total_guesses_medium),
           ng[3],"{:.5f}".format(ng[3]/total_guesses_large))

/home/yusoi/Desktop/mmdetection/mmdet/datasets/coco.py:698: UserWarning: The key "bbox" is deleted for more accurate mask AP of small/medium/large instances since v2.12.0. This does not change the overall mAP calculation.
  warnings.warn(


Loading and preparing results...
DONE (t=0.08s)
creating index...
index created!


/home/yusoi/Desktop/mmdetection/mmdet/datasets/coco.py:589: RuntimeWarning: invalid value encountered in ulong_scalars
  iou = np.sum(intersection)/np.sum(union)


TP 0.48182 0.62619 0.52591 0.48131 0.46329 0.40000 0.50214 0.46293
FP 0.04310 0.15060 0.08331 0.04265 0.04896 0.10000 0.09659 0.04851
TN 0.42193 0.15982 0.29424 0.42334 0.43630 0.37500 0.30783 0.43753
FN 0.05315 0.06339 0.09654 0.05270 0.05144 0.12500 0.09343 0.05104
P 0.64825 0.91874 0.63863 0.90510
R 0.63392 0.89843 0.63387 0.89835
F1 0.63850 0.90492 0.63307 0.89723
CG 8218 0.70559 4 0.02139 82 0.31907 8132 0.72588
IG 870 0.07470 1 0.00535 33 0.12840 836 0.07462
NG 2559 0.21971 182 0.97326 142 0.55253 2235 0.19950


/home/yusoi/anaconda3/envs/pytorch/lib/python3.9/tempfile.py:817: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpyd9vhn54'>
  _warnings.warn(warn_message, ResourceWarning)


# Image Viewer

In [56]:
with open('work_dirs/ensemble_aux/coco_image_names.pickle', 'rb') as handle:
    images_names = pickle.load(handle)
with open('work_dirs/ensemble_aux/coco_image_names2.pickle', 'rb') as handle:
    images_names2 = pickle.load(handle)

In [27]:
GT = dataset.gt_return()
p = GT
print(len(p[139]))

2


In [20]:
print(GT[139][0]['segmentation'])

{'size': [426, 640], 'counts': b'iP\\52W=100O10000O010O10O01N100fNL^E6a:N[E2e:1WE0XOIX;;XE1]OGZ;?eD7N^O\\;e1O00000O10001O:F0001N2O1O1OmE_N^8_1oFXOP9b2O2O06H2N2N2N2N2N3PNTF<n9BVF:l9DYF7j9FZF7o9_OUF=S:[OQFa0W:TOPFh0^;K7H8H9G[SX2'}


In [14]:
result_list = ["GT","results/1|2|3|4_e=average_c=0.2_v=-1_d=0.4_a=0.pkl","results/1|2_e=average_c=0.2_v=-1_d=0.4_a=-1.pkl"]

In [89]:
from ipywidgets import Select,Layout,Output,VBox,HBox
from IPython.display import display
import os
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

config, checkpoint,_,_ = model_dict[0][1]
model = init_detector(config, checkpoint, device='cuda:0')

select = Select(
            description="Image",
            options=images_names,
            value=images_names[0],
            rows=15,
            disabled=False,
        )

select2 = Select(
            description="Ensemble",
            options=result_list,
            value=result_list[0],
            rows=15,
            disabled=False,
        )
output = Output(
            layout=Layout(width='70%')
        )


display(HBox([VBox([select,select2]),output]))

def on_element_clicked(b):
    with output:
        output.clear_output()
        if select2.value != "GT":
            with open(select2.value,"r"):
                results = mmcv.load(select2.value)
                img = images_names2[select.value]
                seg_res = []
                for i in range(0,len(results[img][1])):
                    seg_res2 = []
                    for j in range(0,len(results[img][1][i])):
                        mask = mask_util.decode(results[img][1][i][j]).astype(bool)
                        seg_res2.append(mask)
                    seg_res.append(seg_res2)
                show_result_pyplot(model, select.value, (results[img][0],seg_res), score_thr=CREDIBILITY_THRESHOLD)
        else:
            results = GT
            img = int(os.path.splitext(select.value)[0].split("/")[-1])
            seg_res = []
            bbox_res = []
            image = Image.open(select.value)
            
            fig, ax = plt.subplots()
            fig.figsize=(10, 8)
            fig.dpi=80
            ax.imshow(image,aspect='auto')
            
            for i in range(0,len(results[img])):
                bbox_res = results[img][i]['bbox']
                bbox = patches.Rectangle((bbox_res[0], bbox_res[1]), bbox_res[2], bbox_res[3], linewidth=1, edgecolor='r', facecolor='none')
                ax.add_patch(bbox)
                mask = mask_util.decode(results[img][i]['segmentation']).astype(int)
                mask = np.ma.masked_values(mask, 0)
                
                ax.imshow(mask, alpha=0.5, cmap="Reds", interpolation = 'none')
                    
            print(bbox_res)
            
            plt.show()
            #plt.imshow(img_2, alpha=0.5)
            #show_result_pyplot(model, select.value, (bbox_res,seg_res), score_thr=CREDIBILITY_THRESHOLD)

select.observe(on_element_clicked)
select2.observe(on_element_clicked)

Use load_from_local loader


In [ ]:
config, checkpoint,_,_ = model_dict[0][1]
model = init_detector(config, checkpoint, device='cuda:0')

for img in range(0,len(dataset)):
    seg_res = []
    for i in range(0,len(results[img][1])):
        seg_res2 = []
        for j in range(0,len(results[img][1][i])):
            mask = mask_util.decode(results[img][1][i][j]).astype(bool)
            seg_res2.append(mask)
        seg_res.append(seg_res2)
        
    
    
    print(dataset[img]['img_metas'][0].data['filename'])
    show_result_pyplot(model, dataset[img]['img_metas'][0].data['filename'], (results[img][0],seg_res), score_thr=CREDIBILITY_THRESHOLD)

        


# Result Recorder

In [63]:
#%%capture cap --no-stderr

import gc
import re
import pickle


def ensemble_and_evaluate(model_dict,order):
    results = run_ensemble(model_dict, CREDIBILITY_THRESHOLD,True,ENSEMBLE_METHOD,result_type='segm')

    title = "results/"+('|'.join(str(e) for e in order))+"_e="+str(ENSEMBLE_METHOD)+"_c="+str(CREDIBILITY_THRESHOLD)+"_v="+str(VIABLE_COUNTABILITY)+"_d="+str(DEVIATION_THRESHOLD)
    if ENSEMBLE_METHOD == 'average':
        title = title + "_a="+str(AVERAGE_ACCEPTABILITY)
    elif ENSEMBLE_METHOD == 'weighted_average':
        title = title + "_a2="+str(AVERAGE_ACCEPTABILITY_2)
    title = title + ".pkl"
    with open(title, 'wb') as f:
        pickle.dump(results, f)

    test_cfg = Config.fromfile(test_config)
    # in case the test dataset is concatenated
    samples_per_gpu = 1
    if isinstance(test_cfg.data.test, dict):
        test_cfg.data.test.test_mode = True
        samples_per_gpu = test_cfg.data.test.pop('samples_per_gpu', 1)
        if samples_per_gpu > 1:
            # Replace 'ImageToTensor' to 'DefaultFormatBundle'
            test_cfg.data.test.pipeline = replace_ImageToTensor(
                test_cfg.data.test.pipeline)
    elif isinstance(test_cfg.data.test, list):
        for ds_cfg in test_cfg.data.test:
            ds_cfg.test_mode = True
        samples_per_gpu = max(
            [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in test_cfg.data.test])
        if samples_per_gpu > 1:
            for ds_cfg in test_cfg.data.test:
                ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

    # init distributed env first, since logger depends on the dist info.
    distributed = False

    #rank, _ = get_dist_info()
    # allows not to create
    #mmcv.mkdir_or_exist(osp.abspath(args.work_dir))
    #timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
    #json_file = osp.join(args.work_dir, f'eval_{timestamp}.json')

    # build the dataloader
    dataset = build_dataset(test_cfg.data.test)
    data_loader = build_dataloader(
        dataset,
        samples_per_gpu=samples_per_gpu,
        workers_per_gpu=test_cfg.data.workers_per_gpu,
        dist=distributed,
        shuffle=False)

    eval_kwargs = dict(metric=["bbox","segm"],classwise=True)
    metric = dataset.evaluate(results, **eval_kwargs)
    bc_info = dataset.evaluateBC(results,0.35)
    
    tps = np.sum(list(bc_info['tp'].values()),axis=0)
    fps = np.sum(list(bc_info['fp'].values()),axis=0)
    tns = np.sum(list(bc_info['tn'].values()),axis=0)
    fns = np.sum(list(bc_info['fn'].values()),axis=0)

    #Copiar por aqui

    total_1 = sum([tps[0],fps[0],tns[0],fns[0]])
    total_1_small = sum([tps[1],fps[1],tns[1],fns[1]])
    total_1_medium = sum([tps[2],fps[2],tns[2],fns[2]])
    total_1_large = sum([tps[3],fps[3],tns[3],fns[3]])
    total_2 = sum([tps[4],fps[4],tns[4],fns[4]])
    total_2_small = sum([tps[5],fps[5],tns[5],fns[5]])
    total_2_medium = sum([tps[6],fps[6],tns[6],fns[6]])
    total_2_large = sum([tps[7],fps[7],tns[7],fns[7]])

    print("True Positives",tps[0],"{:.5f}".format(tps[0]/total_1),
                           tps[1],"{:.5f}".format(tps[1]/total_1_small),
                           tps[2],"{:.5f}".format(tps[2]/total_1_medium),
                           tps[3],"{:.5f}".format(tps[3]/total_1_large),
                           tps[4],"{:.5f}".format(tps[4]/total_2),
                           tps[5],"{:.5f}".format(tps[5]/total_2_small),
                           tps[6],"{:.5f}".format(tps[6]/total_2_medium),
                           tps[7],"{:.5f}".format(tps[7]/total_2_large))
    print("False Positives",fps[0],"{:.5f}".format(fps[0]/total_1),
                            fps[1],"{:.5f}".format(fps[1]/total_1_small),
                            fps[2],"{:.5f}".format(fps[2]/total_1_medium),
                            fps[3],"{:.5f}".format(fps[3]/total_1_large),
                            fps[4],"{:.5f}".format(fps[4]/total_2),
                            fps[5],"{:.5f}".format(fps[5]/total_2_small),
                            fps[6],"{:.5f}".format(fps[6]/total_2_medium),
                            fps[7],"{:.5f}".format(fps[7]/total_2_large))
    print("True Negatives",tns[0],"{:.5f}".format(tns[0]/total_1),
                           tns[1],"{:.5f}".format(tns[1]/total_1_small),
                           tns[2],"{:.5f}".format(tns[2]/total_1_medium),
                           tns[3],"{:.5f}".format(tns[3]/total_1_large),
                           tns[4],"{:.5f}".format(tns[4]/total_2),
                           tns[5],"{:.5f}".format(tns[5]/total_2_small),
                           tns[6],"{:.5f}".format(tns[6]/total_2_medium),
                           tns[7],"{:.5f}".format(tns[7]/total_2_large))
    print("False Negatives",fns[0],"{:.5f}".format(fns[0]/total_1),
                            fns[1],"{:.5f}".format(fns[1]/total_1_small),
                            fns[2],"{:.5f}".format(fns[2]/total_1_medium),
                            fns[3],"{:.5f}".format(fns[3]/total_1_large),
                            fns[4],"{:.5f}".format(fns[4]/total_2),
                            fns[5],"{:.5f}".format(fns[5]/total_2_small),
                            fns[6],"{:.5f}".format(fns[6]/total_2_medium),
                            fns[7],"{:.5f}".format(fns[7]/total_2_large))

    cg = list(bc_info['correct_guesses'].values())
    ig = list(bc_info['incorrect_guesses'].values())
    ng = list(bc_info['not_guessed'].values())
    correct_guesses = np.sum(cg,axis=0)
    incorrect_guesses = np.sum(ig,axis=0)
    not_guessed = np.sum(ng,axis=0)

    is_crowds = list(bc_info['is_crowd'].values())
    crowd_cg = list(zip(cg,is_crowds))
    crowd_cg,_ = zip(*[x for x in crowd_cg if x[1] is True])
    crowd_ig = list(zip(ig,is_crowds))
    crowd_ig,_ = zip(*[x for x in crowd_ig if x[1] is True])
    crowd_ng = list(zip(ng,is_crowds))
    crowd_ng,_ = zip(*[x for x in crowd_ng if x[1] is True])
    crowd_correct_guesses = np.sum(crowd_cg,axis=0)
    crowd_incorrect_guesses = np.sum(crowd_ig,axis=0)
    crowd_not_guessed = np.sum(crowd_ng,axis=0) 
        
    total_guesses = sum([correct_guesses[0],incorrect_guesses[0],not_guessed[0]])
    total_guesses_small = sum([correct_guesses[1],incorrect_guesses[1],not_guessed[1]])
    total_guesses_medium = sum([correct_guesses[2],incorrect_guesses[2],not_guessed[2]])
    total_guesses_large = sum([correct_guesses[3],incorrect_guesses[3],not_guessed[3]])
    total_crowd_guesses = sum([crowd_correct_guesses[0],crowd_incorrect_guesses[0],crowd_not_guessed[0]])
    total_crowd_guesses_small = sum([crowd_correct_guesses[1],crowd_incorrect_guesses[1],crowd_not_guessed[1]])
    total_crowd_guesses_medium = sum([crowd_correct_guesses[2],crowd_incorrect_guesses[2],crowd_not_guessed[2]])
    total_crowd_guesses_large = sum([crowd_correct_guesses[3],crowd_incorrect_guesses[3],crowd_not_guessed[3]])
    
    print("Correct Guesses",correct_guesses[0],"{:.5f}".format(correct_guesses[0]/total_guesses),
                            correct_guesses[1],"{:.5f}".format(correct_guesses[1]/total_guesses_small),
                            correct_guesses[2],"{:.5f}".format(correct_guesses[2]/total_guesses_medium),
                            correct_guesses[3],"{:.5f}".format(correct_guesses[3]/total_guesses_large),
                            crowd_correct_guesses[0],"{:.5f}".format(crowd_correct_guesses[0]/total_crowd_guesses),
                            crowd_correct_guesses[1],"{:.5f}".format(crowd_correct_guesses[1]/total_crowd_guesses_small),
                            crowd_correct_guesses[2],"{:.5f}".format(crowd_correct_guesses[2]/total_crowd_guesses_medium),
                            crowd_correct_guesses[3],"{:.5f}".format(crowd_correct_guesses[3]/total_crowd_guesses_large))
    print("Incorrect Guesses",incorrect_guesses[0],"{:.5f}".format(incorrect_guesses[0]/total_guesses),
                              incorrect_guesses[1],"{:.5f}".format(incorrect_guesses[1]/total_guesses_small),
                              incorrect_guesses[2],"{:.5f}".format(incorrect_guesses[2]/total_guesses_medium),
                              incorrect_guesses[3],"{:.5f}".format(incorrect_guesses[3]/total_guesses_large),
                              crowd_incorrect_guesses[0],"{:.5f}".format(crowd_incorrect_guesses[0]/total_crowd_guesses),
                              crowd_incorrect_guesses[1],"{:.5f}".format(crowd_incorrect_guesses[1]/total_crowd_guesses_small),
                              crowd_incorrect_guesses[2],"{:.5f}".format(crowd_incorrect_guesses[2]/total_crowd_guesses_medium),
                              crowd_incorrect_guesses[3],"{:.5f}".format(crowd_incorrect_guesses[3]/total_crowd_guesses_large))
    print("Not Guessed",not_guessed[0],"{:.5f}".format(not_guessed[0]/total_guesses),
                        not_guessed[1],"{:.5f}".format(not_guessed[1]/total_guesses_small),
                        not_guessed[2],"{:.5f}".format(not_guessed[2]/total_guesses_medium),
                        not_guessed[3],"{:.5f}".format(not_guessed[3]/total_guesses_large),
                        crowd_not_guessed[0],"{:.5f}".format(crowd_not_guessed[0]/total_crowd_guesses),
                        crowd_not_guessed[1],"{:.5f}".format(crowd_not_guessed[1]/total_crowd_guesses_small),
                        crowd_not_guessed[2],"{:.5f}".format(crowd_not_guessed[2]/total_crowd_guesses_medium),
                        crowd_not_guessed[3],"{:.5f}".format(crowd_not_guessed[3]/total_crowd_guesses_large))
    

def ordering_recursion(models, missing_iterations, used_array, order_array):
    if missing_iterations == 0:
        ordered_model_dict = []
        print("Model Order: ",end="")
        for i in range(0,len(order_array)):
            ordered_model_dict.append(models[order_array[i]])
            print(ordered_model_dict[i][0],end=" -> ")
        print("")
        ensemble_and_evaluate(ordered_model_dict,order_array)
    else:
        for i in range(0, len(models)):
            if not used_array[i]:
                cur_used_array = used_array.copy()
                cur_used_array[i] = True
                cur_order_array = order_array.copy()
                cur_order_array.append(i)
                ordering_recursion(models,missing_iterations-1,cur_used_array,cur_order_array)

def non_ordered_recursion(models, next_model, missing_iterations, used_array, order_array):       
    if missing_iterations == 0:
        ordered_model_dict = []
        print("Model Order: ",end="")
        for i in range(0,len(order_array)):
            ordered_model_dict.append(models[order_array[i]])
            print(ordered_model_dict[i][0],end=" -> ")
        print("")
        ensemble_and_evaluate(ordered_model_dict,order_array)
    else:
        for i in range(next_model, len(models)):
            if not used_array[i]:
                cur_used_array = used_array.copy()
                cur_used_array[i] = True
                cur_order_array = order_array.copy()
                cur_order_array.append(i)
                non_ordered_recursion(models,i,missing_iterations-1,cur_used_array,cur_order_array)
                
def ensemble_permutations(models):
    used_array = [False] * len(models)
    for i in range(1,len(models)+1):
        non_ordered_recursion(models,0, i, used_array, [])
        #ordering_recursion(models, i, used_array, [])

In [64]:
perm_results = ensemble_permutations(model_dict)

Model Order: mask_rcnn_r50_fpn_1x_cityscapes -> 
Loading inference results from model: mask_rcnn_r50_fpn_1x_cityscapes


/home/yusoi/Desktop/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  warnings.warn(


Use load_from_local loader
([array([[386.69058   ,  77.156815  , 498.77228   , 345.02853   ,
          0.99998236]], dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32), array([], shape=(0, 5), dtype=float32)], [[{'size': [427, 640], 'counts': b'^`R5=i<a0@9G5N2_KROaLQ1Y3VOdLk0Z3YOcLi0Z3ZOWLT1h3oNoKX1n3nNRKZOUNo1g6kNhJU2W5mMcJX2\\5jMVJc2i5`MjIk2U6XMbIn2^6P2N2M2O11N1N3M2O1O1L3M301N3K4K6M3M3M3K5M2N3J6I6N2OM2^Ob0N4O102N10001O3L4M3M2N5K7I4L2N7I;E6J4L7I5K3M2N1O1O1O1O3M1O2N00000001O0mLdKWM]4Y2YL`Mh3W2dLeM]3Q2oLkMT3`1dMXNa2c1fMVN_2e1hMUN[2i1nMmMW2P2f4K5\\MlEi1l:O2N2O1N2O1O1N2O0O3L3N2N1O3L4L3M2N3M6I5L4KdQk1'}], [], [], [], [], [], [], []])


 27%|██████████                            | 1326/5000 [00:01<00:05, 692.43it/s]


KeyboardInterrupt: 